In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

n = 2
r = 2
k = 2
log_path = f'{n}_{r}_{k}_resultados.log'
results_path = f'{n}_{r}_{k}_resultados.txt'

In [ ]:
n_processes = [2,4,8,16,32]
r=2
k=2
execution_times = list()
for n in n_processes:
    log_path = f'{n}_{r}_{k}_resultados.log'
    results_path = f'{n}_{r}_{k}_resultados.txt'
    log_frame = pd.read_csv(log_path, sep=';')
    log_frame.columns = ['timestamp', 'id', 'action']
    log_frame['timestamp'] = pd.to_datetime(log_frame['timestamp'], unit='s')
    log_frame.sort_values(by='timestamp', inplace=True)

    results_frame = pd.read_csv(results_path, sep=';')
    results_frame.columns = ['timestamp', 'id']
    results_frame['timestamp'] = pd.to_datetime(results_frame['timestamp'], format='%Y-%m-%d %H:%M:%S.%f')
    results_frame.sort_values(by='timestamp', inplace=True)

    #Checks if the log is coeherent with the results
    acquire_coeherent = (log_frame.loc[log_frame['action']=='aquire', 'id'] == results_frame['id']).all()
    release_coeherent = (log_frame.loc[log_frame['action']=='release', 'id'] == results_frame['id']).all()
    is_correct = acquire_coeherent and release_coeherent
    print(f'{n}. {is_correct}')

    last_id = None
    last_action = None
    log_wrong_data = list()
    # Checking if an acquirer was made without release
    for idx, row in log_frame.iterrows():

        if (last_id is not None) and (last_id != row['id']) and (last_action is not None) and (last_action == row['action']):
            #wrong_idxs = [idx-1, idx, idx+1]
            log_wrong_data.append(idx-1)
            log_wrong_data.append(idx)
            log_wrong_data.append(idx+1)

    print(f'{n}. {log_frame.loc[log_wrong_data]}')
    if not is_correct or not log_frame.empty:
        raise ValueError('Log and results are not coeherent')
    execution_times.append(results_frame.loc[-1]['timestamp'] - results_frame.loc[0]['timestamp'])

In [ ]:
plt.subplots(1,1)
plt.plot(n_processes, execution_times)
plt.xlabel('Número de processos')
plt.ylabel('Tempo de execução')
plt.title(f'Tempo de execução para r={r} e k={k}')
plt.savefig(f'{r}_{k}_plot.png', dpi=200)